# Behavioral Design Patterns

Focus on how classes distribute responsibilities among them,
and at the same time each class just does a single cohesive function.
It is like a F1 Pits Team, each one has a single Responsability, but
all together creates a complete team workflow.


## Iterator

This pattern allows sequential
access to the elements of an aggregate object without exposing its
underlying representation.

In [4]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)


class SyntaxTreeIterator:
    def __init__(self, root):
        self.stack = [root]

    def __iter__(self):
        return self

    def __next__(self):
        if not self.stack:
            raise StopIteration

        node = self.stack.pop()
        self.stack.extend(reversed(node.children))
        return node.value


# Create a sample syntax tree
root = Node("Root")
node1 = Node("+")
node2 = Node("2")
node3 = Node("5")
node4 = Node("=")
node5 = Node("7")

root.add_child(node1)
root.add_child(node4)
node1.add_child(node2)
node1.add_child(node3)
node4.add_child(node5)

# Traverse the syntax tree using the iterator pattern
iterator = SyntaxTreeIterator(root)
for value in iterator:
    print(value)


Root
+
2
5
=
7


## Memento

This pattern lets you save and
restore the previous state of an object without revealing the details of
its implementation.

In [7]:
class CodeEditor:
    def __init__(self):
        self.code = ""

    def write_code(self, code): #setState
        self.code += code

    def save(self):
        return CodeMemento(self.code)

    def restore(self, memento):
        self.code = memento.get_saved_code()


class CodeMemento:
    def __init__(self, code):
        self.saved_code = code

    def get_saved_code(self):
        return self.saved_code


class CodeVersionControl:
    def __init__(self):
        self.saved_versions = []

    def save_version(self, memento):
        self.saved_versions.append(memento)

    def get_version(self, index):
        return self.saved_versions[index]


# ================= CLIENT
editor = CodeEditor()
version_control = CodeVersionControl()

editor.write_code("print('Hello, World!')")
version_control.save_version(editor.save()) # 0

editor.write_code("\nprint('This is a code subversion example')")
version_control.save_version(editor.save()) # 1

editor.write_code("\nprint('Using the Memento pattern')")
version_control.save_version(editor.save()) # 2

# Restore to a previous version
editor.restore(version_control.get_version(2))

# Print the current code
print(editor.code)


print('Hello, World!')
print('This is a code subversion example')
print('Using the Memento pattern')


## Strategy

This pattern lets you define a family of algorithms, put each of them into a separate class, and make their objects interchangeable

In [8]:
from abc import ABC, abstractmethod

# Define the abstract strategy class
class SortingStrategy(ABC):
    @abstractmethod
    def sort(self, data):
        pass

# Define concrete strategy classes
class BubbleSortStrategy(SortingStrategy):
    def sort(self, data):
        print("Sorting using Bubble Sort")
        # Implementation of Bubble Sort algorithm

class QuickSortStrategy(SortingStrategy):
    def sort(self, data):
        print("Sorting using Quick Sort")
        # Implementation of Quick Sort algorithm

class MergeSortStrategy(SortingStrategy):
    def sort(self, data):
        print("Sorting using Merge Sort")
        # Implementation of Merge Sort algorithm

# Define the context class
class Sorter:
    def __init__(self, strategy: SortingStrategy):
        self.strategy = strategy

    def set_strategy(self, strategy: SortingStrategy):
        self.strategy = strategy

    def sort_data(self, data):
        self.strategy.sort(data)

# Usage
data = [5, 2, 8, 1, 9]

sorter = Sorter(BubbleSortStrategy())
sorter.sort_data(data)

sorter.set_strategy(QuickSortStrategy())
sorter.sort_data(data)

sorter.set_strategy(MergeSortStrategy())
sorter.sort_data(data)


Sorting using Bubble Sort
Sorting using Quick Sort
Sorting using Merge Sort


## Template

This pattern deﬁnes the program skeleton of an algorithm in the superclass but lets subclasses override speciﬁc steps of the algorithm without changing its structure.

In [9]:
from abc import ABC, abstractmethod

class PastaDish(ABC):
    def __prepare_pasta(self):
        print("Boiling water and cooking pasta")

    @abstractmethod
    def add_sauce(self):
        pass    

    @abstractmethod
    def add_toppings(self):
        pass

    def __serve(self):
        print("Serve the pasta dish")

    def make_recipe(self):
        self.__prepare_pasta()
        self.add_sauce()
        self.add_toppings()
        self.__serve()

class SpaghettiMeatballs(PastaDish):
    def add_sauce(self):
        print("Adding tomato sauce")

    def add_toppings(self):
        print("Adding meatballs and grated cheese")

class RavioliAlfredo(PastaDish):
    def add_sauce(self):
        print("Adding Alfredo sauce")

    def add_toppings(self):
        print("Adding Parmesan cheese and parsley")

# Usage
spaghetti_meatballs = SpaghettiMeatballs()
spaghetti_meatballs.make_recipe()
print()
ravioli_alfredo = RavioliAlfredo()
ravioli_alfredo.make_recipe()


Boiling water and cooking pasta
Adding tomato sauce
Adding meatballs and grated cheese
Serve the pasta dish

Boiling water and cooking pasta
Adding Alfredo sauce
Adding Parmesan cheese and parsley
Serve the pasta dish


## Chain of Responsability

This patterns lets you pass requests along a chain of handlers. Upon receiving a
request, each handler decides either to process the request or to pass it along the chain.

In [13]:
class Email:
    def __init__(self, subject, body):
        self.subject = subject
        self.body = body


class SpamFilter:
    def __init__(self, next_filter=None):
        self.next_filter = next_filter

    def process(self, email):
        if self.is_spam(email):
            print("Email classified as spam")
        elif self.next_filter:
            self.next_filter.process(email)
        else:
            print("Email classified as not spam")

    def is_spam(self, email):
        # Implement your spam classification logic here
        return False


class KeywordFilter(SpamFilter):
    def is_spam(self, email):
        if "buy now" in email.subject.lower() or "limited time offer" in email.body.lower():
            return True
        return False


class SenderFilter(SpamFilter):
    def is_spam(self, email):
        if email.subject.startswith("SPAM:") or email.body.startswith("SPAM:"):
            return True
        return False


# Usage
email = Email("Important Ofer", "SPAM: Limited time ofer! Buy now!")
spam_filter = KeywordFilter(SenderFilter())
spam_filter.process(email)


Email classified as spam


## State

This _pattern_  lets an object alter its behavior when its internal state changes. It appears as if the object changed its class.

In [3]:
from abc import ABC, abstractmethod

class VendingMachineState(ABC):
    @abstractmethod
    def insert_coin(self, amount):
        pass

    @abstractmethod
    def select_product(self, product):
        pass

    @abstractmethod
    def dispense_product(self):
        pass

    @abstractmethod
    def return_change(self):
        pass


class NoCoinState(VendingMachineState):
    def insert_coin(self, amount):
        print(f"Inserted {amount} coins")
        return CoinInsertedState()

    def select_product(self, product):
        print("Please insert coins first")

    def dispense_product(self):
        print("Please insert coins first")

    def return_change(self):
        print("No coins to return")


class CoinInsertedState(VendingMachineState):
    def __init__(self):
        self.total_amount = 0

    def insert_coin(self, amount):
        print(f"Inserted {amount} coins")
        self.total_amount += amount

    def select_product(self, product):
        if self.total_amount >= product.price:
            print(f"Selected {product.name}")
            return ProductSelectedState(product)
        else:
            print("Insufficient coins")

    def dispense_product(self):
        print("Please select a product first")

    def return_change(self):
        print(f"Returning {self.total_amount} coins")
        self.total_amount = 0
        return NoCoinState()


class ProductSelectedState(VendingMachineState):
    def __init__(self, product):
        self.product = product

    def insert_coin(self, amount):
        print("Cannot insert coins after product selection")

    def select_product(self, product):
        print("Product already selected")

    def dispense_product(self):
        print(f"Dispensing {self.product.name}")
        self.product.reduce_quantity()
        return NoCoinState()

    def return_change(self):
        print("Cannot return change after product selection")


class Product:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity

    def reduce_quantity(self):
        if self.quantity > 0:
            self.quantity -= 1
            print(f"{self.name} dispensed")
        else:
            print(f"{self.name} is out of stock")


class VendingMachine:
    def __init__(self):
        self.state = NoCoinState()

    def insert_coin(self, amount):
        self.state = self.state.insert_coin(amount)

    def select_product(self, product):
        self.state.select_product(product)

    def dispense_product(self):
        self.state = self.state.dispense_product()

    def return_change(self):
        self.state = self.state.return_change()


# Usage
product1 = Product("Coke", 1.5, 5)
product2 = Product("Chips", 1.0, 3)

vending_machine = VendingMachine()

vending_machine.select_product(product1)  # Please insert coins first

vending_machine.insert_coin(1)  # Inserted 1 coins
vending_machine.select_product(product1)  # Insufficient coins

vending_machine.insert_coin(2)  # Inserted 2 coins
vending_machine.select_product(product1)  # Selected Coke
vending_machine.dispense_product()  # Dispensing Coke

vending_machine.insert_coin(1)  # Inserted 1 coins
vending_machine.select_product(product2)  # Selected Chips
vending_machine.dispense_product()  # Dispensing Chips

vending_machine.dispense_product()  # Please select a product first

vending_machine.return_change()  # Returning 1 coins


Please insert coins first
Inserted 1 coins
Insufficient coins
Inserted 2 coins


AttributeError: 'NoneType' object has no attribute 'select_product'

# Mediator

This pattern lets you reduce chaotic dependencies between objects. The pattern restricts direct communications between the objects and forces them to collaborate only via a mediator object.

In [1]:
from abc import ABC, abstractmethod


class Mediator(ABC):
    @abstractmethod
    def notify(self, sender, event):
        pass


class SmartHome(Mediator):
    def __init__(self):
        self.sensors = []
        self.actors = []

    def add_sensor(self, sensor):
        self.sensors.append(sensor)

    def add_actor(self, actor):
        self.actors.append(actor)

    def notify(self, sender, event):
        if sender in self.sensors:
            print(f"Sensor {sender.name} detected {event}")
            # Perform actions based on the event
            if event == "motion":
                for actor in self.actors:
                    actor.turn_on_lights()
            elif event == "temperature":
                for actor in self.actors:
                    actor.adjust_thermostat()
        elif sender in self.actors:
            print(f"Actor {sender.name} received {event}")
            # Perform actions based on the event
            if event == "doorbell":
                for sensor in self.sensors:
                    sensor.detect_motion()


class Sensor:
    def __init__(self, name, mediator):
        self.name = name
        self.mediator = mediator

    def detect_motion(self):
        self.mediator.notify(self, "motion")

    def detect_temperature_change(self):
        self.mediator.notify(self, "temperature")


class Actor:
    def __init__(self, name, mediator):
        self.name = name
        self.mediator = mediator

    def turn_on_lights(self):
        self.mediator.notify(self, "lights_on")

    def adjust_thermostat(self):
        self.mediator.notify(self, "thermostat_adjusted")


# Usage
smart_home = SmartHome()

sensor1 = Sensor("Motion Sensor", smart_home)
sensor2 = Sensor("Temperature Sensor", smart_home)
smart_home.add_sensor(sensor1)
smart_home.add_sensor(sensor2)

actor1 = Actor("Light Bulb", smart_home)
actor2 = Actor("Thermostat", smart_home)
smart_home.add_actor(actor1)
smart_home.add_actor(actor2)

sensor1.detect_motion()  # Motion Sensor detected motion, turns on lights
sensor2.detect_temperature_change()  # Temperature Sensor detected temperature change, adjusts thermostat


Sensor Motion Sensor detected motion
Actor Light Bulb received lights_on
Actor Thermostat received lights_on
Sensor Temperature Sensor detected temperature
Actor Light Bulb received thermostat_adjusted
Actor Thermostat received thermostat_adjusted


## Command

This pattern turns a request into a stand-alone object that contains all information about the request.

In [6]:
from abc import ABC, abstractmethod

# Document class
class Document:
    def __init__(self, content):
        self.content = content

    def update_content(self, new_content):
        self.content = new_content

    def save(self):
        print("Document saved.")

# Abstract Command class
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

    @abstractmethod
    def undo(self):
        pass

# Concrete SaveCommand class
class SaveCommand(Command):
    def __init__(self, document):
        self.document = document

    def execute(self):
        self.document.save()

    def undo(self):
        pass

# Concrete UpdateCommand class
class UpdateCommand(Command):
    def __init__(self, document, new_content):
        self.document = document
        self.new_content = new_content
        self.previous_content = None

    def execute(self):
        self.previous_content = self.document.content
        self.document.update_content(self.new_content)

    def undo(self):
        self.document.update_content(self.previous_content)

# Concrete RedoUpdateCommand class
class RedoUpdateCommand(Command):
    def __init__(self, document, new_content):
        self.document = document
        self.new_content = new_content
        self.previous_content = None

    def execute(self):
        self.previous_content = self.document.content
        self.document.update_content(self.new_content)

    def undo(self):
        self.document.update_content(self.previous_content)

# Usage
document = Document("Initial content")

# Create commands
save_command = SaveCommand(document)
update_command = UpdateCommand(document, "Updated content")
redo_update_command = RedoUpdateCommand(document, "Redo updated content")

# Execute commands
save_command.execute()
update_command.execute()
print(document.content)  # Output: "Updated content"

# Undo update command
update_command.undo()
print(document.content)  # Output: "Initial content"

# Redo update command
redo_update_command.execute()
print(document.content)  # Output: "Redo updated content"


Document saved.
Updated content
Initial content
Redo updated content


# Observer

This pattern lets you define a subscription mechanism to notify multiple objects about any events that happen to the object they’re observing.

In [2]:
class Blog:
    def __init__(self):
        self.subscribers = []

    def subscribe(self, subscriber):
        self.subscribers.append(subscriber)

    def unsubscribe(self, subscriber):
        self.subscribers.remove(subscriber)

    def notify_subscribers(self, new_entry):
        for subscriber in self.subscribers:
            subscriber.update(new_entry)


class User:
    def __init__(self, name):
        self.name = name

    def update(self, new_entry):
        print(f"User {self.name} received a notification for a new blog entry: {new_entry}")


# Usage
blog = Blog()

user1 = User("Alice")
user2 = User("Bob")
user3 = User("Charlie")

blog.subscribe(user1)
blog.subscribe(user2)
blog.subscribe(user3)

blog.notify_subscribers("New blog entry: Introduction to Python Decorators")


User Alice received a notification for a new blog entry: New blog entry: Introduction to Python Decorators
User Bob received a notification for a new blog entry: New blog entry: Introduction to Python Decorators
User Charlie received a notification for a new blog entry: New blog entry: Introduction to Python Decorators
